https://www.datacamp.com/community/tutorials/customer-life-time-value

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import datetime as dt
import numpy as np

plt.style.use('seaborn')

In [2]:
data = pd.read_excel("Online Retail.xlsx")

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [16]:
uk_data=data[(data.Country=='United Kingdom')&(data.Quantity>0)][['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]
uk_data['TotalPurchase'] = uk_data['Quantity']*uk_data['UnitPrice']
uk_data['month_yr'] = uk_data['InvoiceDate'].apply(lambda x: x.strftime('%b-%Y'))
print(uk_data.shape)
uk_data.head()

(486286, 7)


,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPurchase,month_yr
0,17850.0,2010-12-01 08:26:00,536365,6,2.55,15.30,Dec-2010
1,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34,Dec-2010
2,17850.0,2010-12-01 08:26:00,536365,8,2.75,22.00,Dec-2010
3,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34,Dec-2010
4,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34,Dec-2010


In [5]:
uk_data_group=uk_data.groupby('CustomerID').agg({'InvoiceDate': lambda date: (date.max() - date.min()).days, 
                                                 'InvoiceNo': lambda freq: len(freq),
                                                 'Quantity': lambda quan: quan.sum(),
                                                 'TotalPurchase': lambda pur: pur.sum()}).reset_index()


uk_data_group.columns = ['CustomerID','num_days','num_transactions','num_units','spent_money']

uk_data_group.head()

,CustomerID,num_days,num_transactions,num_units,spent_money
0,12346.0,0,1,74215,77183.60
1,12747.0,366,103,1275,4196.01
2,12748.0,372,4596,25748,33719.73
3,12749.0,209,199,1471,4090.88
4,12820.0,323,59,722,942.34


In [6]:
# Average Order Value = Total Revenue / Total Number of Orders
uk_data_group['avg_order_value'] = round(uk_data_group['spent_money'] / uk_data_group['num_transactions'], 2)

# Purchase Frequency =  Total Number of Orders / Total Number of Customers
uk_data_group['purchase_freq'] = round(uk_data_group['num_transactions']/uk_data_group['CustomerID'].nunique() , 2)

Purchase Frequency(PF): Purchase Frequency is the ratio of the total number of orders and the total number of customer. It represents the average number of orders placed by each customer.

In [25]:
purchase_frequency = round(sum(uk_data_group['num_transactions'])/uk_data_group.shape[0], 4)

Repeat Rate: Repeat rate can be defined as the ratio of the number of customers with more than one order to the number of unique customers. Example: If you have 10 customers in a month out of who 4 come back, your repeat rate is 40%.

In [26]:
repeat_rate = round(uk_data_group[uk_data_group.num_transactions > 1].shape[0]/uk_data_group.shape[0], 4)

Churn Rate: Churn Rate is the percentage of customers who have not ordered again.

In [29]:
churn_rate = round(1 - repeat_rate, 4)

In [30]:
print(purchase_frequency, repeat_rate, churn_rate)

90.3711 0.9819 0.0181


In [31]:
uk_data_group['profit_margin'] = round(uk_data_group['spent_money'] / uk_data_group['num_transactions'] * 0.05, 2) # assuming only 5% margin
uk_data_group['CLV']=(uk_data_group['avg_order_value']*purchase_frequency)/churn_rate
uk_data_group['cust_lifetime_value']=uk_data_group['CLV']*uk_data_group['profit_margin']
uk_data_group.head()

,CustomerID,num_days,num_transactions,num_units,spent_money,avg_order_value,purchase_freq,profit_margin,CLV,cust_lifetime_value
0,12346.0,0,1,74215,77183.60,77183.60,0.00,3859.18,3.853683e+08,1.487206e+12
1,12747.0,366,103,1275,4196.01,40.74,0.03,2.04,2.034099e+05,4.149561e+05
2,12748.0,372,4596,25748,33719.73,7.34,1.17,0.37,3.664773e+04,1.355966e+04
3,12749.0,209,199,1471,4090.88,20.56,0.05,1.03,1.026536e+05,1.057332e+05
4,12820.0,323,59,722,942.34,15.97,0.02,0.80,7.973627e+04,6.378902e+04
